# Network Rail Land Cover Analyses

This notebook is used to produce annual Network Rail (NR) outputs:

1. Each year the latest land cover 10m land cover map should be clipped by a 1km buffer (i.e., 500m either side of land owned by NR).  
2. A summary of habitats contained within land owned by network rail, orgainsed into NR Maintenance Delivery Units (MDUs).

Some comments are included but processing steps are mostly self explanatory.

This notebook is version x1.0_0.  

Import the libraries

In [ ]:
import gc
import rasterio
import rasterio.plot
import pyproj
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from osgeo import gdal
from osgeo import ogr
import tkinter as tk
from tkinter import filedialog
import os
import numpy as np

Choose a directory for your results

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()

filepath = filedialog.askdirectory(parent=root) + '/'

root.destroy()

print(filepath)

#make a temp and an output directory beneath this
tempdir = filepath + 'temp'
if not os.path.exists(tempdir):
    os.makedirs(tempdir)
resultsdir = filepath + 'results'
if not os.path.exists(resultsdir):
    os.makedirs(resultsdir)

Specify the network rail ownership shape file.  This file will be supplied by NR.  Before proceeding with analyses confirm that you have the correct file.

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()


#shapefile = filedialog.askopenfilename(initialdir=filepath,multiple=False)
shapefile = filedialog.askopenfilename(
    initialdir=filepath, multiple=False,
    filetypes=(
        ("Shapefiles", "*.shp"),
        ("All Files", "*.*")
    )
)

root.destroy()
print (shapefile)

Specify the land cover map

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()

lcmfile = filedialog.askopenfilename(
    initialdir=filepath, multiple=False,
    filetypes=(
        ("Tiff files", "*.tif"),
        ("All Files", "*.*")
    )
)

root.destroy()
print (lcmfile)

Create a 1km buffer about Network Rail's land

In [ ]:
ownership = gpd.read_file(shapefile)
buffered = ownership.buffer(500)
buffered.plot()

Create a raster from the buffered vector.  This will be used to truncate the LCM.

In [ ]:
buffered.to_file(f"{tempdir}/buffered.shp", driver='ESRI Shapefile')

Create a binary mask: 1 for the buffer, 0 for everything else.  Note, allTouched=True includes any raster that is touched by a geometry.

In [ ]:
NoData_value = 0
buffer1k = tempdir + "/buffered_1k.tif"
shape_file = tempdir + "/buffered.shp"
ds = gdal.Rasterize(buffer1k, shape_file, xRes=10, yRes=10, 
                    burnValues=1, outputBounds=[0.0, 0.0, 700000, 1300000], allTouched=True,
                    outputType=gdal.GDT_Byte,noData=NoData_value)
ds = None

Read the LCM and the buffer into memory.  Multiply the buffer by the LCM.  This will 'clip' the LCM to the NR buffer.  Be patient :)!  This might be a good time to have a cup of tea.

In [ ]:
raster_a = rasterio.open(lcmfile).read()
raster_b = rasterio.open(buffer1k).read()
raster_c = raster_a * raster_b
#You'll need to try and free a bit of memory after that monster multiplication
del raster_a
del raster_b
del buffered
gc.collect

raster_c is an array.  We need to convert this into a raster and write it to a file.
For this we need:
driver, width, height, count (=num bands), dtype (datatype) crs, and transform 

In [ ]:
driver = "GTiff"
print ("driver = ", driver)
dim = raster_c.shape
print ("dim = ", dim)
height = dim[1]
print ("height = ", height)
width = dim[2]
print ("width = ", width)
nbands = dim[0] 
print ("nbands = ", nbands)
datatype = raster_c.dtype
print("dtype = ", datatype)
from rasterio.crs import CRS
crs = CRS.from_epsg(27700)
print ("crs = ", crs);
from rasterio.transform import from_origin
transform = from_origin(0,1300000,10,10)
print ("transform = ", transform)

In [ ]:
head, tail = os.path.split(lcmfile)
with rasterio.open(resultsdir + "/clipped_" + tail, "w",
                  driver=driver,
                  height=height,
                  width=width,
                  dtype=datatype,
                  count=nbands,
                  crs=crs,
                  transform=transform) as dst:
    dst.write(raster_c)

Free up the memory

In [ ]:
del dst
del raster_c
gc.collect

Calculate the zonal statistics. The next step will take quite a while. Use this time gift to delete some emails

In [ ]:
from rasterstats import zonal_stats
cmap = {1: '_01_dec', 2: '_02_con', 3: '_03_ara', 4: '_04_imp', 5: '_05_neu', 6: '_06_cal', 7: '_07_aci', 8: '_08_fen', 9: '_09_hea', 10: '_10_heg', 11: '_11_bog', 12: '_12_inl', 13: '_13_swa', 14: '_14_fwa', 15: '_15_slr', 16: '_16_sls', 17: '_17_lro', 18: '_18_lse', 19: '_19_sma', 20: '_20_urb', 21: '_21_sub'}
zs = zonal_stats(vectors=ownership['geometry'], raster=lcmfile, categorical=True, category_map=cmap)

Export zonal statistics to a shapefile by Attaching the pixel counts to each object

In [ ]:
import pandas as pd
stats = pd.DataFrame(zs).fillna(0)
stats = stats.reindex(sorted(stats.columns), axis=1)
results = pd.merge(left=ownership, right=stats, how='left', left_index=True, right_index=True)
head, tail = os.path.split(shapefile)
results.to_file(resultsdir + '/' + 'pixel_count_' + tail)

Group the zonal statistics and export to a CSV

In [ ]:
summary = results.groupby(["REGION_NAM","ROUTE_NAME", "MDU_NAME"]).agg({"_01_dec":np.sum, "_02_con":np.sum, "_03_ara":np.sum, "_04_imp":np.sum, "_05_neu":np.sum,
                                                             "_06_cal":np.sum, "_07_aci":np.sum, "_08_fen":np.sum, "_09_hea":np.sum, "_10_heg":np.sum,
                                                             "_11_bog":np.sum, "_12_inl":np.sum, "_13_swa":np.sum, "_14_fwa":np.sum, "_15_slr":np.sum,
                                                             "_16_sls":np.sum, "_17_lro":np.sum, "_18_lse":np.sum, "_19_sma":np.sum, "_20_urb":np.sum,
                                                             "_21_sub":np.sum})
summary

Now give birth to a CSV table

In [ ]:
head, tail = os.path.split(lcmfile)
csv = resultsdir + "/mdu_summmary_" + os.path.splitext(tail)[0]+".csv"
summary.to_csv(csv)

You have finished. Well done!